<a href="https://colab.research.google.com/github/Giffy/MongoDB_PyMongo_Tutorial/blob/master/1_2_Bicing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example 1 - Bicing stations

##1. Install Pymongo

In [4]:
!pip install pymongo==3.7.2      # Install PyMongo
!pip install folium geopandas    # Install folium for map visualization

##2. Import libraries

In [0]:
import pymongo                            # Library to access MongoDB
from pymongo import MongoClient           # Imports MongoClient 
import pandas as pd                       # Library to work with dataframes
import folium                             # Library to visualize a map

##3. Connect to database

In [0]:
# uri (uniform resource identifier) defines the connection parameters 
# uri = 'mongodb:// USER : PASSWORD @ SERVER_NAME : PORT / DATABASENAME')
# uri = 'mongodb:// USER : PASSWORD @ SERVER_NAME : PORT / DATABASE_NAME, CLUSTER_1_NAME : PORT , CLUSTER_2_NAME : PORT')
uri = 'localhost:27017'
# start client to connect to MongoDB server 
client = MongoClient( uri )

##4. Download data

In [7]:
dataset = "https://www.bicing.cat/availability_map/getJsonObject"     # Get JSON file from bicing
!wget $dataset

--2018-11-21 10:09:36--  https://www.bicing.cat/availability_map/getJsonObject
Resolving www.bicing.cat (www.bicing.cat)... 62.97.112.138
Connecting to www.bicing.cat (www.bicing.cat)|62.97.112.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘getJsonObject’

getJsonObject           [   <=>              ] 117.58K   249KB/s    in 0.5s    

2018-11-21 10:09:37 (249 KB/s) - ‘getJsonObject’ saved [120405]



##5. Load data to database

In [8]:
!mongodb-linux-x86_64-debian71-3.0.15/bin/mongoimport --db barcelona --collection bicing --jsonArray /content/getJsonObject

2018-11-21T10:09:39.129+0000	connected to: localhost
2018-11-21T10:09:40.046+0000	imported 463 documents


In [0]:
db = client.barcelona                     # Set the database to work on
db.list_collection_names()                # List the collections available
collection = db.bicing                    # Collection alias

##6. Quick data overview

In [10]:
num_documents = collection.count_documents({'_id' : {'$exists' : 1}})     # Counts the documents in database
print ( 'Number of documents in database = ' + str(num_documents) )
list ( collection.find().limit(1) )                                       # Shows the first document

Number of documents in database = 463


[{'_id': ObjectId('5bf52ee30f50e03768521161'),
  'address': 'Roger de Flor/ Gran Vía',
  'addressNumber': '126',
  'bikes': '0',
  'district': '2',
  'id': '2',
  'lat': '41.39553',
  'lon': '2.17706',
  'name': '02 - C/ ROGER DE FLOR, 126',
  'nearbyStations': '360,368,387,414',
  'slots': '25',
  'stationType': 'BIKE',
  'status': 'OPN',
  'zip': '08010'}]

##7. Query to database:  Get active stations with at least 3 bicycles

In [0]:
# Loading database query in pandas Dataframe
filters = {'status':'OPN', 'bikes' : {'$gte' : '3' }}   # Usage of gte Query Operator  $gte = "greater than or equal"
fields = { '_id', 'lat' , 'lon', 'bikes', 'slots'}

query = list( collection.find( filters , fields ) )
df = pd.DataFrame ( query )                             # Load the database reply in a Pandas DataFrame

In [0]:
print ( 'Numer of active stations with at least 3 bicycles: ' + str(len (query)) )

Numer of active stations with at least 3 bicycles: 124


In [0]:
df.iloc[0] # prints the first DataFrame row 

_id      5bf3ccfb69e41ab55efb2c0d
bikes                           4
lat                     41.391349
lon                       2.18061
slots                          34
Name: 0, dtype: object

##8. Mark Bicing stations in map

In [0]:
center_lat = 41.394055
center_lon = 2.181299

locationmap = folium.Map(location=[ center_lat , center_lon ], zoom_start=16 ) #, width=600, height=400 )

longitud  = len( df )
for i in range ( longitud ):
    lng = float(df.iloc[i][3])
    lat = float(df.iloc[i][2])
    description = 'Bikes: ' + str(df.iloc[i][1]) + '<br> Empty slots: ' + str(df.iloc[i][4])
    folium.Marker( [ lat , lng ],
                 popup= description,
                 icon=folium.Icon(color='red')).add_to(locationmap)

locationmap